In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Types of models that will be used to test the methods
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
# Scoring metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [2]:
training_data = pd.read_csv('./datasets/Train_Data.csv', index_col=0)
testing_data = pd.read_csv('./datasets/Test_Data.csv', index_col=0)
training_data.head()

,avg_trip_distance_hour,avg_fare_amount_hour,avg_tolls_amount_hour,avg_tip_amount_hour,avg_Brooklyn_PU_hour,avg_Manhattan_PU_hour,avg_Queens_PU_hour,avg_Staten_Island_PU_hour,avg_Bronx_DO_hour,avg_Brooklyn_DO_hour,...,avg_Queens_DO_hour,avg_Staten_Island_DO_hour,avg_Mins_In_Ride_hour,Year,Month,Day,Hour,Minute,Second,avg_total_amount_hour
2580,2.236724,9.381992,0.000000,1.243851,0.028736,0.932950,0.022989,0.0,0.038314,0.024904,...,0.028736,0.0,8.662280,2020,4,17,13,0,0,13.582953
2534,2.240061,9.397930,0.000000,1.381967,0.014344,0.948770,0.024590,0.0,0.043033,0.026639,...,0.024590,0.0,8.702090,2020,4,15,15,0,0,13.496905
1653,2.469164,10.333324,0.000353,2.063430,0.010463,0.963948,0.024581,0.0,0.005546,0.059876,...,0.035926,0.0,10.659279,2020,3,9,22,0,0,16.207518
4281,2.099069,9.251151,0.000000,1.604006,0.022082,0.955836,0.020505,0.0,0.012618,0.031546,...,0.034700,0.0,8.363486,2020,6,27,10,0,0,13.726811
3057,2.511803,10.052960,0.004345,1.380512,0.037951,0.922201,0.017078,0.0,0.036053,0.039848,...,0.022770,0.0,9.328482,2020,5,7,10,0,0,13.604888


In [4]:
training_data = training_data.fillna(0)
testing_data = testing_data.fillna(0)

In [5]:
testing_data.isnull().sum()

avg_trip_distance_hour       0
avg_fare_amount_hour         0
avg_tolls_amount_hour        0
avg_tip_amount_hour          0
avg_Brooklyn_PU_hour         0
avg_Manhattan_PU_hour        0
avg_Queens_PU_hour           0
avg_Staten_Island_PU_hour    0
avg_Bronx_DO_hour            0
avg_Brooklyn_DO_hour         0
avg_Manhattan_DO_hour        0
avg_Queens_DO_hour           0
avg_Staten_Island_DO_hour    0
avg_Mins_In_Ride_hour        0
Year                         0
Month                        0
Day                          0
Hour                         0
Minute                       0
Second                       0
avg_total_amount_hour        0
dtype: int64

In [6]:
print(training_data.shape)
testing_data.shape

(3056, 21)


(1310, 21)

In [7]:
# splitting the data into training and testing variables
X_train = training_data.drop(['avg_total_amount_hour'], axis=1)
X_test = testing_data.drop(['avg_total_amount_hour'], axis=1)
y_train = training_data[['avg_total_amount_hour']]
y_test = testing_data[['avg_total_amount_hour']]

In [8]:
# This first model will test on the raw data using Linear Regression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg_y_pred = linreg.predict(X_test)

In [9]:
# Returns RMSE
train_linreg_RMSE = mean_squared_error(y_train, linreg.predict(X_train))
r2_linreg_train = r2_score(y_train, linreg.predict(X_train))
linreg_RMSE = mean_squared_error(y_test, linreg_y_pred, squared=False)
linreg_r2 = r2_score(y_test, linreg_y_pred)
linreg_r2

0.6764275253566046

In [10]:
linreg_parameters = {'RMSE_train': train_linreg_RMSE, 'RMSE_test': linreg_RMSE, 
                    'r2_train': r2_linreg_train, 'r2_test': linreg_r2}

# Ridge Regression

In [11]:
ridge = Ridge()
params = {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]}
ridge_cv = GridSearchCV(ridge, param_grid=params, cv=5)
ridge_cv.fit(X_train, y_train)
ridge_cv.best_params_

{'alpha': 0.1}

In [12]:
ridgereg = Ridge(0.1)
ridgereg.fit(X_train, y_train)
ridgereg_y_pred = ridgereg.predict(X_test)

In [13]:
train_ridgereg_RMSE = mean_squared_error(y_train, ridgereg.predict(X_train))
r2_ridgereg_train = r2_score(y_train, ridgereg.predict(X_train))
ridgereg_RMSE = mean_squared_error(y_test, ridgereg_y_pred, squared=False)
ridgereg_r2 = r2_score(y_test, ridgereg_y_pred)
ridgereg_r2

0.6775786470340428

In [14]:
ridge_parameters = {'RMSE_train': train_ridgereg_RMSE, 'RMSE_test': ridgereg_RMSE, 
                    'r2_train': r2_ridgereg_train, 'r2_test': ridgereg_r2, 'alpha': 0.1}

# Lasso Regression

In [15]:
lasso = Lasso()
params = {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]}
lasso_cv = GridSearchCV(lasso, param_grid=params, cv=5)
lasso_cv.fit(X_train, y_train)
lasso_cv.best_params_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 475.43109280683257, tolerance: 0.7659248757514227
  positive)


{'alpha': 0.001}

In [16]:
lassoreg = Lasso(alpha=0.001)
lassoreg.fit(X_train, y_train)
lassoreg_y_pred = lassoreg.predict(X_test)

In [17]:
train_lassoreg_RMSE = mean_squared_error(y_train, lassoreg.predict(X_train))
r2_lassoreg_train = mean_squared_error(y_train, lassoreg.predict(X_train))
lassoreg_RMSE = mean_squared_error(y_test, lassoreg_y_pred, squared=False)
lassoreg_r2 = r2_score(y_test, lassoreg_y_pred)
lassoreg_r2

0.6805918317425883

In [18]:
lasso_parameters = {'RMSE_train': train_lassoreg_RMSE, 'RMSE_test': lassoreg_RMSE, 
                    'r2_train': r2_lassoreg_train, 'r2_test': lassoreg_r2, 'alpha': 0.001}

# KNearest Neighbors

In [19]:
knn = KNeighborsRegressor()
params = {'n_neighbors': np.arange(1, 10)}
knn_cv = GridSearchCV(knn, param_grid=params, cv=5)
knn_cv.fit(X_train, y_train)
knn_cv.best_params_

{'n_neighbors': 8}

In [20]:
k_nn = KNeighborsRegressor(n_neighbors=8)
k_nn.fit(X_train, y_train)
k_nn_y_pred = k_nn.predict(X_test)

In [21]:
train_k_nn_RMSE = mean_squared_error(y_train, k_nn.predict(X_train))
r2_k_nn_train = r2_score(y_train, k_nn.predict(X_train))
k_nn_RMSE = mean_squared_error(y_test, k_nn_y_pred, squared=False)
k_nn_r2 = r2_score(y_test, k_nn_y_pred)
k_nn_r2

0.7531360334651717

In [22]:
k_nn_parameters = {'RMSE_train': train_k_nn_RMSE, 'RMSE_test': k_nn_RMSE, 
                    'r2_train': r2_k_nn_train, 'r2_test': k_nn_r2, 'n_neighbors': 8}

# Decision Tree

In [23]:
decisiontree = DecisionTreeRegressor()
params = {'max_depth': np.arange(1, 10)}
decisiontree = GridSearchCV(decisiontree, param_grid=params, cv=5)
decisiontree.fit(X_train, y_train)
decisiontree.best_params_

{'max_depth': 7}

In [24]:
tree = DecisionTreeRegressor(max_depth=7)
tree.fit(X_train, y_train)
tree_y_pred = tree.predict(X_test)

In [25]:
train_tree_RMSE = mean_squared_error(y_train, tree.predict(X_train))
r2_tree_train = r2_score(y_train, tree.predict(X_train))
tree_RMSE = mean_squared_error(y_test, tree_y_pred, squared=False)
tree_r2 = r2_score(y_test, tree_y_pred)
tree_RMSE

0.8328893575768286

In [26]:
tree_parameters = {'RMSE_train': train_tree_RMSE, 'RMSE_test': tree_RMSE, 
                    'r2_train': r2_tree_train, 'r2_test': tree_r2, 'max_depth':7}

# Random Forest Regressor

In [27]:
rfr = RandomForestRegressor()
params = {'n_estimators': np.arange(1, 10),
         'max_depth': np.arange(1, 10)}
rfr_cv = GridSearchCV(rfr, param_grid=params, cv=5)
rfr_cv.fit(X_train, y_train)
rfr_cv.best_params_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y wa

{'max_depth': 9, 'n_estimators': 9}

In [28]:
rfrtrees = RandomForestRegressor(n_estimators=5, max_depth=9)
rfrtrees.fit(X_train, y_train)
rfrtrees_y_pred = rfrtrees.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [29]:
train_rfrtrees_RMSE = mean_squared_error(y_train, rfrtrees.predict(X_train))
r2_rfrtrees_train = r2_score(y_train, rfrtrees.predict(X_train))
rfrtrees_RMSE = mean_squared_error(y_test, rfrtrees_y_pred, squared=False)
rfrtrees_r2 = r2_score(y_test, rfrtrees_y_pred)
rfrtrees_r2

0.8514804953371156

In [30]:
rfrtrees_parameters = {'RMSE_train': train_rfrtrees_RMSE, 'RMSE_test': rfrtrees_RMSE, 
                    'r2_train': r2_rfrtrees_train, 'r2_test': rfrtrees_r2, 'n_estimators': 5, 'max_depth': 9}

# Gradient Boosting

In [31]:
gradboost = GradientBoostingRegressor()
params = {'n_estimators': np.arange(1, 10),
         'max_depth': np.arange(1, 10)}
gradboost_cv = GridSearchCV(gradboost, param_grid=params, cv=5)
gradboost_cv.fit(X_train, y_train)
gradboost_cv.best_params_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

{'max_depth': 9, 'n_estimators': 9}

In [32]:
gradientboosting = GradientBoostingRegressor(n_estimators=9, max_depth=9)
gradientboosting.fit(X_train, y_train)
gradientboosting_y_predict = gradientboosting.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [33]:
train_gradientboosting_RMSE = mean_squared_error(y_train, gradientboosting.predict(X_train))
r2_gradientboosting_train = r2_score(y_train, gradientboosting.predict(X_train))
gradientboosting_RMSE = mean_squared_error(y_test, gradientboosting_y_predict, squared=False)
gradientboosting_r2 = r2_score(y_test, gradientboosting_y_predict)
gradientboosting_r2

0.7343410127898181

In [34]:
gradientboosting_parameters = {'RMSE_train': train_gradientboosting_RMSE, 'RMSE_test': gradientboosting_RMSE, 
                               'r2_train': r2_gradientboosting_train, 'r2_test': gradientboosting_r2,
                              'n_estimators': 9, 'max_depth': 9}

# Ada Boost

In [35]:
adaBoost = AdaBoostRegressor()
params = {'n_estimators': np.arange(1, 10)}
adaBoost_cv = GridSearchCV(adaBoost, param_grid=params, cv=5)
adaBoost_cv.fit(X_train, y_train)
adaBoost_cv.best_params_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

{'n_estimators': 5}

In [36]:
adaBoosting = AdaBoostRegressor(n_estimators=6)
adaBoosting.fit(X_train, y_train)
adaBoosting_y_pred = adaBoosting.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [37]:
train_adaBoosting_RMSE = mean_squared_error(y_train, adaBoosting.predict(X_train))
r2_adaBoosting_train = r2_score(y_train, adaBoosting.predict(X_train))
adaBoosting_RMSE = mean_squared_error(y_test, adaBoosting_y_pred)
adaBoosting_r2 = r2_score(y_test, adaBoosting_y_pred)
adaBoosting_r2

0.7518858582753265

In [38]:
adaboosting_parameters = {'RMSE_train': train_adaBoosting_RMSE, 'RMSE_test': adaBoosting_RMSE, 
                          'r2_train': r2_adaBoosting_train, 'r2_test': adaBoosting_r2, 'n_estimators': 6}

In [39]:
df = pd.DataFrame([linreg_parameters, ridge_parameters, lasso_parameters, k_nn_parameters, tree_parameters,
                  rfrtrees_parameters, gradientboosting_parameters, adaboosting_parameters],
                 index=['Linear Regression', 'Ridge Regression', 'Lasso Regression', 'KNearestNeighbors',
                       'Single Decison Tree', 'Random Forest', 'Gradient Boosting', 'AdaBoosting'])
df

,RMSE_train,RMSE_test,r2_train,r2_test,alpha,n_neighbors,max_depth,n_estimators
Linear Regression,1.268635,1.040149,0.604423,0.676428,NaN,NaN,NaN,NaN
Ridge Regression,1.275546,1.038297,0.602268,0.677579,0.100,NaN,NaN,NaN
Lasso Regression,1.296831,1.033434,1.296831,0.680592,0.001,NaN,NaN,NaN
KNearestNeighbors,0.749192,0.908528,0.766392,0.753136,NaN,8.0,NaN,NaN
Single Decison Tree,0.392672,0.832889,0.877560,0.792530,NaN,NaN,7.0,NaN
Random Forest,0.246644,0.704695,0.923093,0.851480,NaN,NaN,9.0,5.0
Gradient Boosting,0.616607,0.942479,0.807734,0.734341,NaN,NaN,9.0,9.0
AdaBoosting,0.785707,0.829604,0.755006,0.751886,NaN,NaN,NaN,6.0


The best performing model is the RandomForest Regressor with max_depth=9 and n_estimators=5.

In [40]:
df.to_csv('Model_Metrics.csv')